# Projeto de Chatbot de atendimento ao cliente baseado em regras

## Projeto desenvolvido para a disciplina de Agentes Conversacionais - IA - PUCPR

### Chatbot baseado em regras

É o tipo de chatbot mais simples que existe. Consiste em um dataset com respostas pré-definidas e uma série de regras que servem para encontrar estas respostas (ou intenções) no dataset.

Apesar das limitações, esses tipos de chatbots podem ser bastante eficientes e produtivos se tiverem regras bem definidas e um dataset substancial de respostas.

### Qual o contexto do chatbot?

Irei desenvolver um chatbot de interação com clientes de um restaurante. O objetivo é que o chatbot auxilie o cliente a encontrar as informações desejadas sem que precise navegar por vários links do site.

### Serão utilizadas as seguintes ferramentas

*   **NLTK** - O toolkit de Processamento de Linguagem Natural em Python.
*   **Expressões Regulares** - o pacote de regex do Python será utilizado para otimizar a busca de padrões.
*   **WordNet** - o grande banco de dados léxico e semântico, disponível em diversos idiomas.


### Construindo o chatbot
O chatbot vai operar da seguinte maneira:

1.   Recebe **entrada** do usuário
2.   Procura **palavras-chave** na entrada do usuário
3.   Define a **intenção associada** à entrada
4.   Obtém a **resposta baseada na intenção** definida
5.   Mostra a **resposta** ao usuário

Portanto, teremos uma **tabela que associe as palavras-chave desejadas, com as intenções definidas** em nosso dataset. A seguir um exemplo:
![Exemplo de busca de intenção e resposta](https://docs.google.com/uc?export=download&id=1LnOTZiahBSv9VGrCSExbylNbxHpclMKZ)

#### Importando as bibliotecas
Importação do pacote de expressões regulares do Python e também o acesso ao WordNet dado pelo NLTK.

In [1]:
import re
import nltk
from nltk.corpus import wordnet
# Download do wordnet
nltk.download('wordnet')
# Open Multilingual Wordnet
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

#### Construindo a lista de palavras-chave
Esta é uma etapa que pode demandar bastante tempo, ainda mais se o desenvolvedor quiser definir as palavras manualmente. Porém, quanto mais palavras-chave tiver, maior a cobertura do chatbot.

In [7]:
# Lista de palavras
palavras = ['olá', 'horário', 'obrigado', 'local', 'cardápio']
# Dicionário de sinônimos
lista_sinonimos = {}

# Percorre a lista de palavras
for palavra in palavras:
  sinonimos = []
  # Então é feita a busca de sinônimos da lista 'palavras' no wordnet e pt-br:
  for syn in wordnet.synsets(palavra, lang='por'):
    # Busca das formas léxicas das palavras e as retorna
    for lem in syn.lemmas(lang='por'):
      # Adiciona a forma na lista
      sinonimos.append(lem.name())

  # Adicionar ao dicionário elementos únicos
  lista_sinonimos[palavra] = set(sinonimos)

print(lista_sinonimos)

{'olá': {'olá'}, 'horário': {'programa', 'ordem_do_dia', 'cronograma', 'plano', 'horário', 'agenda'}, 'obrigado': {'apreciativo', 'obrigado', 'agradecido'}, 'local': {'local', 'lugar', 'posição', 'situação'}, 'cardápio': {'cardápio', 'menu'}}


In [12]:
# Vou adicionar um sinônimo a lista de sinônimos
lista_sinonimos['olá'].add('oi')
lista_sinonimos['obrigado'].add('muito obrigado')
lista_sinonimos['obrigado'].add('valeu')
lista_sinonimos['obrigado'].add('agradeço')
lista_sinonimos['local'].add('localização')
lista_sinonimos['local'].add('localidade')
lista_sinonimos['local'].add('endereço')
lista_sinonimos['local'].add('atendem')

print(lista_sinonimos)

{'olá': {'oi', 'olá'}, 'horário': {'programa', 'ordem_do_dia', 'cronograma', 'plano', 'horário', 'agenda'}, 'obrigado': {'apreciativo', 'muito obrigado', 'valeu', 'obrigado', 'agradeço', 'agradecido'}, 'local': {'localização', 'lugar', 'posição', 'local', 'endereço', 'situação', 'localidade', 'atendem'}, 'cardápio': {'cardápio', 'menu'}}


#### Construindo dicionário de intenções
Vou utilizar o dicionário de sinônimos construído e mapeá-los para as possíveis intenções. Além disso, vou formatar as palavras-chave de modo que sejam interpretadas pela ferramenta de busca de expressões regulares.

In [16]:
# Dicionário de palavras-chave e intenções:
keywords = {}
keywords_dict = {}

# Criando um registro de intenção para saudações
keywords['saudacao'] = []
# Adiciona sinônimos da palavra-chave 'olá' à entrada recém criada
for sin in list(lista_sinonimos['olá']):
  keywords['saudacao'].append('.*\\b'+sin+'\\b.*')
  # print(keywords)

# Criando um registro de intanção para saudação de bom dia
keywords['saudacao_bom_dia'] = ['bom dia']

# Criando um registro de intanção para saudação de boa tarde
keywords['saudacao_boa_tarde'] = ['boa tarde']

# Criando um registro de intanção para saudação de boa noite
keywords['saudacao_boa_noite'] = ['boa noite']

# Criando um registro de intanção para agradecimento
keywords['agradecimento'] = []
# Adiciona sinônimos da palavra-chave 'obrigado' à entrada recém criada
for sin in list(lista_sinonimos['obrigado']):
  keywords['agradecimento'].append('.*\\b'+sin+'\\b.*')

# Criando um registro de intanção para localização
keywords['localizacao'] = []
# Adiciona sinônimos da palavra-chave 'obrigado' à entrada recém criada
for sin in list(lista_sinonimos['local']):
  keywords['localizacao'].append('.*\\b'+sin+'\\b.*')

# Agora cria-se um novo registro de intenção para horário de atendimento
keywords['horario_atendimento'] = []
# Adiciona sinônimos da palavra-chave 'horário' à entrada recém criada
for sin in list(lista_sinonimos['horário']):
  keywords['horario_atendimento'].append('.*\\b'+sin+'\\b.*')
  # print(keywords)

# Agora cria-se um novo registro de intenção para o cardápio
keywords['cardapio'] = []
# Adiciona sinônimos da palavra-chave 'horário' à entrada recém criada
for sin in list(lista_sinonimos['cardápio']):
  keywords['cardapio'].append('.*\\b'+sin+'\\b.*')

for intent, keys in keywords.items():
  # Vou unir todas as palavras-chave sinônimas com  o operador 'OU'
  keywords_dict[intent] = re.compile('|'.join(keys))

print(keywords_dict)

{'saudacao': re.compile('.*\\boi\\b.*|.*\\bolá\\b.*'), 'saudacao_bom_dia': re.compile('bom dia'), 'saudacao_boa_tarde': re.compile('boa tarde'), 'saudacao_boa_noite': re.compile('boa noite'), 'agradecimento': re.compile('.*\\bapreciativo\\b.*|.*\\bmuito obrigado\\b.*|.*\\bvaleu\\b.*|.*\\bobrigado\\b.*|.*\\bagradeço\\b.*|.*\\bagradecido\\b.*'), 'localizacao': re.compile('.*\\blocalização\\b.*|.*\\blugar\\b.*|.*\\bposição\\b.*|.*\\blocal\\b.*|.*\\bendereço\\b.*|.*\\bsituação\\b.*|.*\\blocalidade\\b.*|.*\\batendem\\b.*'), 'horario_atendimento': re.compile('.*\\bprograma\\b.*|.*\\bordem_do_dia\\b.*|.*\\bcronograma\\b.*|.*\\bplano\\b.*|.*\\bhorário\\b.*|.*\\bagenda\\b.*'), 'cardapio': re.compile('.*\\bcardápio\\b.*|.*\\bmenu\\b.*')}


#### Definindo as respostas
Agora vou definir as respostas para cada intenção. É importante sempre definir uma intenção padrão, para ser acionada caso não achemos nenhuma palavra-chave na entrada do usuário.

In [14]:
# Dicionário de respostas
respostas = {
    'saudacao' : 'Olá. Como posso ajudar?',
    'saudacao_bom_dia' : 'Bom dia! Como posso ajudar?',
    'saudacao_boa_tarde' : 'Boa tarde! Como posso ajudar?',
    'saudacao_boa_noite' : 'Boa noite! Como posso ajudar?',
    'horario_atendimento' : 'Nosso horário de atendimento é de segunda a sexta, das 11h às 23h',
    'padrao' : 'Desculpe, não entendi o que você disse. Poderia reformular a pergunta?',
    'agradecimento' : 'De nada! Se precisar de mais alguma coisa, estou à disposição.',
    'localizacao' : 'Estamos localizados na Travessa do Tranco, nº 9',
    'cardapio' : 'Claro, segue nosso cardápio ...'
}

É possível adicionar mais respostas (intenções) ao dicionário, como agradecimento, despedida e, até localização:

**agradecimento:** 'De nada! Se precisar de mais alguma coisa, estou à disposição.

**despedida:** 'Até logo! Tenha um ótimo dia.

**localizacao:** 'Estamos localizados na Rua Exemplo, 123, no centro da cidade.'

#### Linkando as intenções e gerando as repostas (front-end)
Aqui será construído de fato o algoritmo que fará a interação com o usuário (o gerenciador de diálogo), onde receberemos uma entrada do mesmo, e utilizando regex, iremos buscar as palavras-chave no texto e associar com a intenção desejada.

In [17]:
print("Bem vindo ao Restaurante As Três Vassouras! ")

# Recebe a entrada até o usuário finalizar a conversa
while True:

  # Recebe a entrada do usuário e a coloca em letras minúsculas
  entrada = input().lower()

  # lista_saida = ['tchau', 'até mais', 'até']
  # for saida in lista_saida:
  #   if saida in entrada:
  #     print('Obrigado pela preferência! Até logo!')
  #     break

  # Condição de saída
  if entrada == 'sair' or entrada == 'até' or entrada == 'até mais' or entrada == 'tchau':
    print('Obrigado pela preferência! Até logo!')
    break

  matched_intent = None

  for intent, pattern in keywords_dict.items():

    # Busca as palavras-chave na entrada do usuário utilizando o regex
    if re.search(pattern, entrada):
      # caso encontre o padrão regex, o salva na intenção correspondente
      matched_intent = intent

  # Padrão
  key = 'padrao'
  if matched_intent in respostas:
    key = matched_intent

  # Resposta correspondente
  print(respostas[key])

Bem vindo ao Restaurante As Três Vassouras! 
Bom dia
Bom dia! Como posso ajudar?
Gostaria do cardápio, por favor
Claro, segue nosso cardápio ...
Obrigado
De nada! Se precisar de mais alguma coisa, estou à disposição.
Onde vocês atendem?
Estamos localizados na Travessa do Tranco, nº 9
Entendi, muito obrigado
De nada! Se precisar de mais alguma coisa, estou à disposição.
Até mais
Obrigado pela preferência! Até logo!
